# Modelling

This part of the project takes the modified dataset from the last step. The previous parts of the project can be found here:
* [Part 1](https://github.com/marcopeix/credit-modelling-lending-club/blob/master/DataCleaning.ipynb)
* [Part 2](https://github.com/marcopeix/credit-modelling-lending-club/blob/master/Feature%20Engineering.ipynb)

**Techniques used:**
* Scikit-learn
* Logistic regression
* Logistic regression with balanced class weight
* Logistic regression with penalty
* Cross-validation
* Random forest classifier

### Introduce the dataset

In [1]:
import pandas as pd
loans = pd.read_csv("cleaned_loans_2007.csv")
print(loans.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38708 entries, 0 to 38707
Data columns (total 38 columns):
loan_amnt                              38708 non-null float64
int_rate                               38708 non-null float64
installment                            38708 non-null float64
emp_length                             38708 non-null int64
annual_inc                             38708 non-null float64
loan_status                            38708 non-null int64
dti                                    38708 non-null float64
delinq_2yrs                            38708 non-null float64
inq_last_6mths                         38708 non-null float64
open_acc                               38708 non-null float64
pub_rec                                38708 non-null float64
revol_bal                              38708 non-null float64
revol_util                             38708 non-null float64
total_acc                              38708 non-null float64
home_ownership_MORTGAGE    

### Define the error metrics
The problem for this project is binary, since a loan is either paid on time or not. Therefore, it is interesting to consider true positive and false positive rates. 

Also, it must be noted that there is a imbalance in the dataset, as most of the loans were paid on time, hence the choice of true positive and false positive rates as error metrics.

### Logistic regression

In [5]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
cols = loans.columns
train_cols = cols.drop("loan_status")
features = loans[train_cols]
target = loans["loan_status"]
lr.fit(features, target)
predictions = lr.predict(features)

### Perform k-fold cross validation to reduce overfitting

In [7]:
from sklearn.model_selection import cross_val_predict

lr = LogisticRegression()
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.9989121566494424
0.9967943009795192


The rates obtained above are not very good, since it is as if the model is predicting that all loans are paid. This is due to class imbalance. Therefore, we must modify the *class_weight* parameter of the model to overcome this issue.

### Logistic regression with balanced class weight

In [8]:
lr = LogisticRegression(class_weight="balanced")
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.6579034841205089
0.38290293855743546


The false positive rate was successfully lowered. However, let's try to reduce it even more.

### Logistic regression with penalty

In [9]:
penalty = {
    0: 10,
    1: 1
}

lr = LogisticRegression(class_weight=penalty)
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.24005076602302602
0.09029385574354408


Although we managed to reduce the false positive rate to 9%, the true positive rate was also significantly lowered. From a conservative point of view, it is reassuring to know that most of the bad investments will be avoided. However, a greater true positive rate would be better.

### Random forest classifier

In [11]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(class_weight="balanced", random_state=1)
predictions = cross_val_predict(rf, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.9708699725017376
0.9271593944790739


Unfortunately, using a random forest classifier does not yield better results. Therefore, it is recommended to use a logistic regression model with penalty for a conservative investor who wishes to fund loans that will be paid on time.